<a href="https://colab.research.google.com/github/pevu97/Jobb-Offer-Classifier/blob/main/notebook/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests

job_id = "machine-learning-engineer-matrix-global-services-Remote"
url = f"https://nofluffjobs.com/api/posting/{job_id}"

headers = {
    "User-Agent": "Mozilla/5.0",
    "Accept": "application/json"
}

response = requests.get(url, headers=headers)

print(response.status_code)
print(response.json())


200
{'id': 'machine-learning-engineer-matrix-global-services-remote', 'apply': {'option': 'email', 'leadCollection': False, 'referenceNumber': '', 'externalPortalLinks': [], 'leadCollectionInfoClause': ''}, 'specs': {'details': {'custom': []}, 'help4Ua': False, 'dailyTasks': ['AI-First Development: Design and implement POCs and production-ready ML/LLM solutions that address real-world business problems.', 'End-to-End Execution: Own projects from research through deployment, including model training, evaluation, integration, and monitoring.', 'Tool Evaluation: Stay current with state-of-the-art ML/LLM tools and frameworks (e.g., LangChain, Hugging Face Transformers, TensorFlow, PyTorch) and recommend their application.', 'Collaboration: Partner with engineering, product, and operations teams to embed AI into user-facing features and internal workflows.', 'System Integration: Build scalable microservices and APIs to deliver AI capabilities across the product.', 'Operationalization: Contr

In [15]:
import requests

url = "https://nofluffjobs.com/api/search/posting"
params = {
    "pageFrom": 1,
    "pageTo": 1,
    "pageSize": 20,
    "salaryCurrency": "PLN",
    "salaryPeriod": "month",
    "region": "pl",
    "language": "pl-PL",
    "withSalaryMatch": "true"
}

headers = {
    "Content-Type": "application/infiniteSearch+json",
    "User-Agent": "Mozilla/5.0"
}

payload = {
    "rawSearch": "requirement=python"
}

response = requests.post(url, headers=headers, params=params, json=payload)

print(response.status_code)
print(response.json())

200
{'criteriaSearch': {'country': [], 'withSalaryMatch': [], 'city': [], 'more': [], 'employment': [], 'requirement': ['Python'], 'salary': [], 'jobPosition': [], 'applicationStatus': [], 'province': [], 'company': [], 'id': [], 'category': [], 'keyword': [], 'jobLanguage': [], 'seniority': []}, 'postings': [{'id': 'python-developer-with-react-square-one-resources-Remote', 'name': 'Square One Resources', 'location': {'places': [{'city': 'Remote', 'url': 'python-developer-with-react-square-one-resources-remote'}, {'country': {'code': 'POL', 'name': 'Poland'}, 'city': 'Warsaw', 'street': 'Puławska 2', 'postalCode': '02-566', 'geoLocation': {'latitude': 52.2129809, 'longitude': 21.0197968}, 'url': 'python-developer-with-react-square-one-resources-warsaw'}, {'country': {'code': 'POL', 'name': 'Poland'}, 'province': 'lower-silesian', 'url': 'python-developer-with-react-square-one-resources-lower-silesian', 'provinceOnly': True}, {'country': {'code': 'POL', 'name': 'Poland'}, 'province': 'k